# Product Matching by Robby Jeffries
## Amazon and Sam's Club

This notebook will identify which products appear in both the Amazon and Sam's Club data sets.

In [122]:
import os
import numpy as np
import pandas as pd

In [123]:
# set working directory
os.chdir('/Users/robbyjeffries/MSEACapstone/Data')

# import raw review data for the Electronics category
amz_raw = pd.read_csv('Products/products_with_first_review_date.csv')
sams_raw = pd.read_excel('Products/Products_Vendor_Start_Date.xlsx') 

In [124]:
sams_raw.head()

,Primary_Desc,Vendor_Name,UPC_Desc,Start Date
0,CG PORT CHARGER LRG,CONCEPT GREEN ENERGY SOLUTIONS,CHARGER,2014-07-01
1,48IN 720P LED TV,JSP LLC,48IN JVC TV,2015-10-14
2,"19"" MTS TV",SANYO MFG CORP,"19""REFURB TV",2014-11-28
3,BELKIN GRP CDY SHEER,BRIGHTSTAR US INC,PHONE CASE,2014-07-03
4,"TOSHIBA 27"" TV",TOSHIBA AMERICA INC,"TV 27"" PIP",2015-05-30


In [125]:
amz_raw.head()

,title,firstReview
0,1000 Cd DVD Silver Aluminum Hard Case for Media Storage Holder w/Hanger Sleeves,2014-01-01
1,Toshiba Satellite L670-17E Laptop Screen 17.3 LED BOTTOM LEFT WXGA++ 1600x900,2014-01-01
2,ASUS GTX780TI-3GD5 Graphics Cards GTX780TI-3GD5,2014-01-01
3,12&quot; SATA 15-Pin Male to Female Power Extension Cable,2014-01-01
4,GreatShield LEAN Series Slim Bluetooth Keyboard Leather Case with Sleep / Wake Cover for Google Nexus 7 FHD 2nd Gen (2013) - Black/Blue,2014-01-01


In [126]:
sams_raw.shape

(1932, 4)

In [127]:
amz_raw.shape

(259873, 2)

In [128]:
new = amz_raw.join(sams_raw['Primary_Desc'])

In [129]:
new = new[['title', 'Primary_Desc']]

In [130]:
new.head()

,title,Primary_Desc
0,1000 Cd DVD Silver Aluminum Hard Case for Media Storage Holder w/Hanger Sleeves,CG PORT CHARGER LRG
1,Toshiba Satellite L670-17E Laptop Screen 17.3 LED BOTTOM LEFT WXGA++ 1600x900,48IN 720P LED TV
2,ASUS GTX780TI-3GD5 Graphics Cards GTX780TI-3GD5,"19"" MTS TV"
3,12&quot; SATA 15-Pin Male to Female Power Extension Cable,BELKIN GRP CDY SHEER
4,GreatShield LEAN Series Slim Bluetooth Keyboard Leather Case with Sleep / Wake Cover for Google Nexus 7 FHD 2nd Gen (2013) - Black/Blue,"TOSHIBA 27"" TV"


In [131]:
#new = new.T.stack().reset_index(drop=True)


In [71]:
all_products = new.to_frame()

In [79]:
# all_products = all_products.rename(columns={0: "product"})

In [80]:
all_products.head()

,product
0,1000 Cd DVD Silver Aluminum Hard Case for Media Storage Holder w/Hanger Sleeves
1,Toshiba Satellite L670-17E Laptop Screen 17.3 LED BOTTOM LEFT WXGA++ 1600x900
2,ASUS GTX780TI-3GD5 Graphics Cards GTX780TI-3GD5
3,12&quot; SATA 15-Pin Male to Female Power Extension Cable
4,GreatShield LEAN Series Slim Bluetooth Keyboard Leather Case with Sleep / Wake Cover for Google Nexus 7 FHD 2nd Gen (2013) - Black/Blue


### Example from Toward Data Science
https://towardsdatascience.com/surprisingly-effective-way-to-name-matching-in-python-1a67328e670e

In [8]:
if False: # Change it to true if you haven't installed it
    !pip install cython
    !pip install git+https://github.com/ing-bank/sparse_dot_topn.git

In [27]:
#  Importing libraries and module and some setting for notebook
import pandas as pd 
import re
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from scipy.sparse import csr_matrix
import sparse_dot_topn.sparse_dot_topn as ct  # Leading Juice for us
import time
pd.set_option('display.max_colwidth', -1)

# reading dataset as df
df = pd.read_excel('Products/Products_Vendor_Start_Date.xlsx') 

# printing first five rows
df.head(5)

/var/folders/c9/z8kjq8qs15n6nq0dr7vdg5640000gn/T/ipykernel_53365/752063784.py:9: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


,Primary_Desc,Vendor_Name,UPC_Desc,Start Date
0,CG PORT CHARGER LRG,CONCEPT GREEN ENERGY SOLUTIONS,CHARGER,2014-07-01
1,48IN 720P LED TV,JSP LLC,48IN JVC TV,2015-10-14
2,"19"" MTS TV",SANYO MFG CORP,"19""REFURB TV",2014-11-28
3,BELKIN GRP CDY SHEER,BRIGHTSTAR US INC,PHONE CASE,2014-07-03
4,"TOSHIBA 27"" TV",TOSHIBA AMERICA INC,"TV 27"" PIP",2015-05-30


In [81]:
# total the number of NaN rows
nan_in_df = all_products.isnull().sum().sum()
  
# printing the number of NaN values present in the whole dataframe
print('Number of NaN values present: ' + str(nan_in_df))

Number of NaN values present: 0


In [83]:
# drop NaN rows
df = all_products.dropna()

In [84]:
def ngrams(string, n=3):
    
    string = re.sub(r'[,-./]|\sBD',r'', string)
    ngrams = zip(*[string[i:] for i in range(n)])
    return [''.join(ngram) for ngram in ngrams]

# Testing ngrams work for verification
print('All 3-grams in "Deluxroom":')
ngrams('Deluxroom')

All 3-grams in "Deluxroom":


['Del', 'elu', 'lux', 'uxr', 'xro', 'roo', 'oom']

In [85]:
products = df['product']
vectorizer = TfidfVectorizer(min_df=1, analyzer=ngrams)
tf_idf_matrix = vectorizer.fit_transform(products)

In [86]:
print(tf_idf_matrix[0])

  (0, 113042)	0.16129878385316632
  (0, 89949)	0.11717223289757034
  (0, 89154)	0.12539077651726988
  (0, 98047)	0.10795626337769408
  (0, 69478)	0.10889387799748038
  (0, 2437)	0.09465047330794478
  (0, 105547)	0.07236638643804462
  (0, 91880)	0.09209309831453706
  (0, 101363)	0.1148872648142515
  (0, 83147)	0.11882858115115032
  (0, 50377)	0.11136714164308963
  (0, 113737)	0.22315196054824632
  (0, 3790)	0.211416541898131
  (0, 105579)	0.10406303301022846
  (0, 87130)	0.09102406710458476
  (0, 97971)	0.11522134265192932
  (0, 103201)	0.10733603970330352
  (0, 50588)	0.12877667860452913
  (0, 1719)	0.09776260383576765
  (0, 87715)	0.10317733055991289
  (0, 91818)	0.09951114242184766
  (0, 82796)	0.1123500219999272
  (0, 106503)	0.13628455387897437
  (0, 103538)	0.13511418521898266
  (0, 110953)	0.08020052540025328
  :	:
  (0, 101889)	0.13978825212280604
  (0, 95200)	0.12811547603306742
  (0, 99705)	0.10589128686776503
  (0, 112079)	0.1271388881044658
  (0, 98606)	0.1325522010845686
  

In [87]:
def awesome_cossim_top(A, B, ntop, lower_bound=0):
    # force A and B as a CSR matrix.
    # If they have already been CSR, there is no overhead
    A = A.tocsr()
    B = B.tocsr()
    M, _ = A.shape
    _, N = B.shape
 
    idx_dtype = np.int32
 
    nnz_max = M*ntop
 
    indptr = np.zeros(M+1, dtype=idx_dtype)
    indices = np.zeros(nnz_max, dtype=idx_dtype)
    data = np.zeros(nnz_max, dtype=A.dtype)
    
    ct.sparse_dot_topn(
            M, N, np.asarray(A.indptr, dtype=idx_dtype),
            np.asarray(A.indices, dtype=idx_dtype),
            A.data,
            np.asarray(B.indptr, dtype=idx_dtype),
            np.asarray(B.indices, dtype=idx_dtype),
            B.data,
            ntop,
            lower_bound,
            indptr, indices, data)
    
    return csr_matrix((data,indices,indptr),shape=(M,N))


In [88]:
#  Top 10 with similarity above 0.8
t1 = time.time()
matches = awesome_cossim_top(tf_idf_matrix, tf_idf_matrix.transpose(), 10, 0.8)
t = time.time()-t1
print("SELFTIMED:", t)

SELFTIMED: 1202.9403960704803


In [181]:
tf_idf_matrix.shape

(261796, 117511)

In [89]:
# unpacks the resulting sparse matrix
def get_matches_df(sparse_matrix, name_vector, top=100):
    non_zeros = sparse_matrix.nonzero()
    
    sparserows = non_zeros[0]
    sparsecols = non_zeros[1]
    
    if top:
        nr_matches = top
    else:
        nr_matches = sparsecols.size
    
    left_side = np.empty([nr_matches], dtype=object)
    right_side = np.empty([nr_matches], dtype=object)
    similairity = np.zeros(nr_matches)
    
    for index in range(0, nr_matches):
        left_side[index] = name_vector[sparserows[index]]
        right_side[index] = name_vector[sparsecols[index]]
        similairity[index] = sparse_matrix.data[index]
    
    return pd.DataFrame({'left_side': left_side,
                          'right_side': right_side,
                           'similairity': similairity})

In [90]:
# store the  matches into new dataframe called matched_df and 
# printing 10 samples
matches_df = get_matches_df(matches, products, top=200)
matches_df = matches_df[matches_df['similairity'] < 0.99999] # For removing all exact matches
matches_df.sample(10)

,left_side,right_side,similairity
181,Kicker 40CWRT102 CompRT Series 10 inch Subwoofer Dual 2 Ohm,Kicker 40CWRT122 CompRT Series 12 inch Subwoofer Dual 2 Ohm,0.896278
90,"Pad Cases,iPad 2 Case,iPad 3 Case,iPad 4 Case,TRAVELLOR[Heavy Duty] iPad Case,Three Layer Armor Defender And Full Body Protective Case Cover With Kickstand And Screen Protector for iPad 2/3/4(Black)","iPad Cases,iPad 2 Case,iPad 3 Case,iPad 4 Case,TRAVELLOR&reg;[Heavy Duty] iPad Case,Three Layer Armor Defender And Full Body Protective Case Cover With Kickstand And Screen Protector for iPad 2/3/4 - Army/Olive",0.885624
25,SwitchEasy Colors Silicone Case for iPod touch 5G (Blue),SwitchEasy Colors Silicone Case for iPod touch 5G (Turquoise),0.807870
193,Nikon D5200 Digital SLR Camera Body (Black) with 18-140mm VR Lens + 32GB Card + Case + Flash + Battery + Filter + Tripod Kit,Nikon D5500 Wi-Fi Digital SLR Camera Body (Black) with 18-200mm VC Lens + 32GB Card + Case + Battery + Tripod + Filter + Kit,0.801593
85,"Pad Cases,iPad 2 Case,iPad 3 Case,iPad 4 Case,TRAVELLOR[Heavy Duty] iPad Case,Three Layer Armor Defender And Full Body Protective Case Cover With Kickstand And Screen Protector for iPad 2/3/4(Black)","iPad Cases,iPad 2 Case,iPad 4 Case,TRAVELLOR[Heavy Duty] iPad Case,Three Layer Armor Defender and Full Body Protective Case Cover with Kickstand and Screen Protector for iPad 2/3/4 - Red",0.924157
36,Okeler Purple 360 Rotating PU Leather Case Cover For ASUS MeMO Pad HD 7 ME173X ME173 with Free Pen,Okeler Red 360 Rotating PU Leather Stand Case Cover For ASUS MeMO Pad HD 7 ME173X ME173 with Free Pen,0.941846
160,"Griffin, iPad mini 1/2/3 Case, Survivor CrossGrip, Protective Gaming Case with shoulder strap, Ergonomic, Rugged, Citron","Griffin, Survivor CrossGrip for iPad mini, Protective Gaming Case with shoulder strap, Ergonomic, Rugged, Black",0.888742
31,Assurant 5-Year Television Protection Plan ($2000-$2249.99),Assurant 5-Year Television Protection Plan ($200-$249.99),0.909246
96,"Pro Digital Hard Lens Hood For The JVC Everio GZ-MG680, MG670, MG630, MG465, MG435, MG365, MG360, MG335, MG330, MG230 High Definition Camcorders","PROFESSIONAL 72 Inch Full Size Tripod with Carrying Case For The JVC Everio GZ-MG680, MG670, MG630, MG465, MG435, MG365, MG360, MG335, MG330, MG230 High Definition Camcorders with Exclusive FREE Complimentary Super Deal Micro Fiber Lens Cleaning Cloth",0.886503
9,GreatShield LEAN Series Slim Bluetooth Keyboard Leather Case with Sleep / Wake Cover for Google Nexus 7 FHD 2nd Gen (2013) - Black/Blue,GreatShield LEAN Series Ultra Slim Bluetooth Keyboard Leather Case with Sleep / Wake Cover for Google Nexus 7 FHD 2nd Gen (2013) - Black/Pink,0.921078


In [174]:
# printing the matches in sorted order
matches_df.sort_values(['similairity'], ascending=False).head(97)

,left_side,right_side,similairity
50,"iPad Air 2 &amp; 1 Bluetooth Keyboard Case, QQ-Tech&reg; Premium Series PU Leather Case Cover W Removable Wireless Keyboard for iPad Air 1st and 2nd [Best iPad Companion] &ndash; Red","iPad Air 2 &amp; 1 Bluetooth Keyboard Case, QQ-Tech&reg; Premium Series PU Leather Case Cover W Removable Wireless Keyboard for iPad Air 1st and 2nd [Best iPad Companion] &ndash; White",0.983058
51,"iPad Air 2 &amp; 1 Bluetooth Keyboard Case, QQ-Tech&reg; Premium Series PU Leather Case Cover W Removable Wireless Keyboard for iPad Air 1st and 2nd [Best iPad Companion] &ndash; Red","iPad Air 2 &amp; 1 Bluetooth Keyboard Case, QQ-Tech&reg; Premium Series PU Leather Case Cover W Removable Wireless Keyboard for iPad Air 1st and 2nd [Best iPad Companion] &ndash; Brown",0.978312
62,"360 Premium Rotating Leather Case Cover Blue Map with Kickstand by KIQ with Kickstand for Apple iPad Air 1 1st Gen 9.7-inch Retina Display 2013 2014 (A1474, A1475, A1476)","360 Premium Rotating Leather Case Cover Black with Kickstand by KIQ with Kickstand for Apple iPad Air 1 1st Gen 9.7-inch Retina Display 2013 2014 (A1474, A1475, A1476)",0.975107
63,"360 Premium Rotating Leather Case Cover Blue Map with Kickstand by KIQ with Kickstand for Apple iPad Air 1 1st Gen 9.7-inch Retina Display 2013 2014 (A1474, A1475, A1476)","360 Premium Rotating Leather Case Cover Map Purple with Kickstand by KIQ with Kickstand for Apple iPad Air 1 1st Gen 9.7-inch Retina Display 2013 2014 (A1474, A1475, A1476)",0.970326
82,"Pad Cases,iPad 2 Case,iPad 3 Case,iPad 4 Case,TRAVELLOR[Heavy Duty] iPad Case,Three Layer Armor Defender And Full Body Protective Case Cover With Kickstand And Screen Protector for iPad 2/3/4(Black)","iPad Cases,iPad 2 Case,iPad 3 Case,iPad 4 Case,TRAVELLOR[Heavy Duty] iPad Case,Three Layer Armor Defender And Full Body Protective Case Cover With Kickstand And Screen Protector for iPad 2/3/4 - Olive",0.957990
83,"Pad Cases,iPad 2 Case,iPad 3 Case,iPad 4 Case,TRAVELLOR[Heavy Duty] iPad Case,Three Layer Armor Defender And Full Body Protective Case Cover With Kickstand And Screen Protector for iPad 2/3/4(Black)","iPad 2 Case,iPad 3 Case,iPad 4 Case,TRAVELLOR[Heavy Duty] iPad Case,Three Layer Armor Defender And Full Body Protective Case Cover With Kickstand And Screen Protector for iPad 2/3/4 - Red",0.956151
117,Elecday Authentic Spider-man Extreme-Duty Military Survivor Silicone Plastic Protective Case Dual Layer water/Dust/Shock/Sand Proof Absorbing Kid-proof With Wall/Glass/Wood Attachable Kids Children Gift Stand Designed For Apple iPad Mini/ Mini2/ Mini 2,Elecday Authentic Spider-man Extreme-Duty Military Survivor Silicone Plastic Protective Case Dual Layer water/Dust/Shock/Sand Proof Absorbing Kid-proof With Wall/Glass/Wood Attachable Kids Children Gift Stand Designed For Apple iPad Mini/ Mini2/ Mini 2 (Olive/olive),0.948375
64,"360 Premium Rotating Leather Case Cover Blue Map with Kickstand by KIQ with Kickstand for Apple iPad Air 1 1st Gen 9.7-inch Retina Display 2013 2014 (A1474, A1475, A1476)","360 Premium Rotating Leather Case Cover Giraffe with Kickstand by KIQ with Kickstand for Apple iPad Air 1 1st Gen 9.7-inch Retina Display 2013 2014 (A1474, A1475, A1476)",0.947737
35,Okeler Purple 360 Rotating PU Leather Case Cover For ASUS MeMO Pad HD 7 ME173X ME173 with Free Pen,Okeler Green 360 Rotating PU Leather Case Cover For ASUS MeMO Pad HD 7 ME173X ME173 with Free Pen,0.947417
84,"Pad Cases,iPad 2 Case,iPad 3 Case,iPad 4 Case,TRAVELLOR[Heavy Duty] iPad Case,Three Layer Armor Defender And Full Body Protective Case Cover With Kickstand And Screen Protector for iPad 2/3/4(Black)","iPad Cases,iPad 2 Case,iPad 3 Case,iPad 4 Case,TRAVELLOR[Heavy Duty] iPad Case,Three Layer Armor Defender And Full Body Protective Case Cover With Kickstand And Screen Protector for iPad 2/3/4 - Gray/White",0.947415


In [173]:
matches_df.shape

(97, 3)

In [93]:
df.shape

(261796, 1)

## Fuzzymatcher library

In [111]:
if False: # Change it to true if you haven't installed it
    !pip install fuzzymatcher
    !pip install pysqlite3

In [112]:
import fuzzymatcher
import pysqlite3

# set working directory
os.chdir('/Users/robbyjeffries/MSEACapstone/Data')

# import raw review data for the Electronics category
amz_raw = pd.read_csv('Products/products_with_first_review_date.csv')
sams_raw = pd.read_excel('Products/Products_Vendor_Start_Date.xlsx') 

sams_raw = sams_raw.dropna()
amz_raw = amz_raw.dropna()

In [113]:
amz_raw.head()

,title,firstReview
0,1000 Cd DVD Silver Aluminum Hard Case for Media Storage Holder w/Hanger Sleeves,2014-01-01
1,Toshiba Satellite L670-17E Laptop Screen 17.3 LED BOTTOM LEFT WXGA++ 1600x900,2014-01-01
2,ASUS GTX780TI-3GD5 Graphics Cards GTX780TI-3GD5,2014-01-01
3,12&quot; SATA 15-Pin Male to Female Power Extension Cable,2014-01-01
4,GreatShield LEAN Series Slim Bluetooth Keyboard Leather Case with Sleep / Wake Cover for Google Nexus 7 FHD 2nd Gen (2013) - Black/Blue,2014-01-01


In [114]:
sams_raw.head()

,Primary_Desc,Vendor_Name,UPC_Desc,Start Date
0,CG PORT CHARGER LRG,CONCEPT GREEN ENERGY SOLUTIONS,CHARGER,2014-07-01
1,48IN 720P LED TV,JSP LLC,48IN JVC TV,2015-10-14
2,"19"" MTS TV",SANYO MFG CORP,"19""REFURB TV",2014-11-28
3,BELKIN GRP CDY SHEER,BRIGHTSTAR US INC,PHONE CASE,2014-07-03
4,"TOSHIBA 27"" TV",TOSHIBA AMERICA INC,"TV 27"" PIP",2015-05-30


In [117]:
# fuzzymatcher.link_table(amz_raw, sams_raw, left_on = 'title', right_on = 'Primary_Desc')

## Fuzzywuzzy

In [119]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

# set working directory
os.chdir('/Users/robbyjeffries/MSEACapstone/Data')

# import raw review data for the Electronics category
amz_raw = pd.read_csv('Products/products_with_first_review_date.csv')
sams_raw = pd.read_excel('Products/Products_Vendor_Start_Date.xlsx') 

sams_raw = sams_raw.dropna()
amz_raw = amz_raw.dropna()

# create lists
amazon_list           = amz_raw['title'].fillna('*').to_list()
sams_list             = sams_raw['Primary_Desc'].fillna('*').to_list()
# validation_list       = hand_typed_list_df['Validation'].fillna('*').to_list()

In [135]:
import pandas as pd
import csv
from fuzzywuzzy import fuzz 
from fuzzywuzzy import process

pd.set_option('display.max_rows', 300)
pd.set_option('display.max_columns', 300)

from tqdm import tqdm_notebook as tqdm

In [144]:
# set working directory
os.chdir('/Users/robbyjeffries/MSEACapstone/Data')

# import raw review data for the Electronics category
amz_raw = pd.read_csv('Products/products_with_first_review_date.csv')
sams_raw = pd.read_excel('Products/Products_Vendor_Start_Date.xlsx') 

amz_sample = amz_raw.sample(100)

sams_raw = sams_raw.dropna()
amz_raw = amz_raw.dropna()

In [170]:
matches = []
for product in tqdm(amz_raw['title']):
    if type(product) != str: # checking base case
        pass
    else:
        match = process.extractOne(product, list(sams_raw['Primary_Desc']))
        if match[1] > 95:
            matches.append(product)
            matches.append(match)

/var/folders/c9/z8kjq8qs15n6nq0dr7vdg5640000gn/T/ipykernel_53365/1629402363.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for product in tqdm(amz_raw['title']):


  0%|          | 0/259873 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [178]:
matches[0]

'Earbud Headphones'

In [179]:
match[1]

86

In [180]:
df = pd.DataFrame({'col':matches})
df.head(100)

,col
0,Earbud Headphones
1,"(EARBUD HEADPHONES , 100)"


## Fuzzy Merge

In [184]:
if False:
    !pip install fuzzy_pandas

In [185]:
import pandas as pd
import fuzzy_pandas as fpd

# set working directory
os.chdir('/Users/robbyjeffries/MSEACapstone/Data')

# import raw review data for the Electronics category
amz_raw = pd.read_csv('Products/products_with_first_review_date.csv')
sams_raw = pd.read_excel('Products/Products_Vendor_Start_Date.xlsx') 

sams_raw = sams_raw.dropna()
amz_raw = amz_raw.dropna()

In [186]:
fpd.fuzzy_merge(amz_raw, sams_raw,
                left_on=['title'],
                right_on=['Primary_Desc'],
                ignore_case=True,
                keep='match')

KeyboardInterrupt: 

In [195]:
import pandas as pd
from fuzzywuzzy import process

process.extractOne('ipad' ,['apple', 'ipad'])

('ipad', 100)

***

# Working Matcher

https://stackoverflow.com/questions/68894999/fuzzy-match-for-2-lists-with-very-similar-names

In [216]:
import pandas as pd
from fuzzywuzzy import process
from tqdm import tqdm

threshold = 90

amz_list = list(amz_raw['title'])

sams_list = list(sams_raw['Primary_Desc'])

response = []

for name_to_find in tqdm(sams_list, total = len(sams_list)):
    resp_match =  process.extractOne(name_to_find, amz_list)
    if resp_match[1] >= threshold:
         row = {'sams_name':name_to_find,'amazon_name':resp_match[0], 'score':resp_match[1]}
         response.append(row)
         print(row)

results = pd.DataFrame(response)

  0%|                                       | 1/1923 [00:40<21:46:50, 40.80s/it]

{'sams_name': 'CG PORT CHARGER LRG ', 'amazon_name': 'Charger', 'score': 90}


  0%|                                       | 6/1923 [03:47<20:06:47, 37.77s/it]

{'sams_name': 'INCIPIO FEATHER     ', 'amazon_name': 'Incipio Feather Case for Microsoft Surface Pro and Surface Pro 2', 'score': 90}


  1%|▏                                     | 11/1923 [06:57<19:26:34, 36.61s/it]

...

{'sams_name': 'DELL INSPIRON 660   ', 'amazon_name': 'Dell Inspiron 660 I660-1044BK Desktop Computer', 'score': 90}


  4%|█▎                                    | 69/1923 [41:22<16:11:02, 31.43s/it]

{'sams_name': 'HARLEY DAVIDSON     ', 'amazon_name': "Hogtunes 0401 Audio Cable (3.3', Connect MP3/GPS to Factory Harley-Davidson Radio)", 'score': 90}


  4%|█▌                                    | 79/1923 [47:05<18:19:15, 35.77s/it]


KeyboardInterrupt: 

In [217]:
results = pd.DataFrame(response)

In [220]:
results.to_csv('CSV_completed/product_match.csv', sep='\t', index=False)

In [202]:
import pandas as pd
import fuzzy_pandas as fpd

# set working directory
os.chdir('/Users/robbyjeffries/MSEACapstone/Data')

# import raw review data for the Electronics category
amz_raw = pd.read_csv('Products/products_with_first_review_date.csv')
sams_raw = pd.read_excel('Products/Products_Vendor_Start_Date.xlsx') 

sams_raw = sams_raw.dropna()
amz_raw = amz_raw.dropna()

In [203]:
sams_raw.head()

,Primary_Desc,Vendor_Name,UPC_Desc,Start Date
0,CG PORT CHARGER LRG,CONCEPT GREEN ENERGY SOLUTIONS,CHARGER,2014-07-01
1,48IN 720P LED TV,JSP LLC,48IN JVC TV,2015-10-14
2,"19"" MTS TV",SANYO MFG CORP,"19""REFURB TV",2014-11-28
3,BELKIN GRP CDY SHEER,BRIGHTSTAR US INC,PHONE CASE,2014-07-03
4,"TOSHIBA 27"" TV",TOSHIBA AMERICA INC,"TV 27"" PIP",2015-05-30


In [211]:
amz_raw['firstReview'] = pd.to_datetime(amz_raw['firstReview'])

In [ ]:
import pandas as pd
from fuzzywuzzy import process
from tqdm import tqdm

threshold = 90

amz_list = list(amz_raw['title'])

sams_list = list(sams_raw['Primary_Desc'])

response = []

for name_to_find in tqdm(sams_list, total = len(sams_list)):
    
    resp_match =  process.extractOne(name_to_find, amz_list)
    if resp_match[1] >= threshold:
         row = {'sams_name':name_to_find,'amazon_name':resp_match[0], 'score':resp_match[1]}
         response.append(row)
         print(row)

results = pd.DataFrame(response)

In [ ]:
amz_raw['title']